# ყველა ფაილის მისამართის ამოღება

In [1]:
!pip install librosa
!pip install pydub

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [3]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.mp3')
    convert_to_wav(mp3_files)
#     m4a_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.m4a')
#     mp4_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp4')
#     aac_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.aac')
#     files = mp3_files + m4a_files + mp4_files + aac_files
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(files) 
#     print(mp3_files)

In [2]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [313]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        zero_crossing_rate = get_zero_crossing_rate(audio)
#         chroma_features = get_chroma_features(audio, sample_rate)
#         other features
#         return [mfccs, zero_crossing_rate] + chroma_features
        return [mfccs, zero_crossing_rate]
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [314]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
#     data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate', 'chroma_features[0]','chroma_features[1]', 'chroma_features[2]', 'class_label'])
    data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate', 'class_label'])
    return data

In [310]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [311]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [285]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
    sp_scaled = np.mean(sp.T,axis=0)
#     sp - (1, სხვადასხვა)
    return sp_scaled

In [287]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    a_scaled = np.mean(a.T,axis=0)
#     a - (1, სხვადასხვა)
    return a_scaled

In [288]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    onset_env_scaled = np.mean(onset_env,axis=0)
#     onset_env - (სხვადასხვა,1)
    return onset_env_scaled

In [289]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [290]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [315]:
data = get_all_features('data')

In [316]:
data

,mfccs,zero_crossing_rate,class_label
0,"[-340.99185, 71.76349, -9.056803, 58.628418, -...",[0.13918022260273974],1
1,"[-549.25867, 23.328505, 2.6170156, 16.299675, ...",[0.3345120337701613],1
2,"[-576.85846, 15.04707, -0.45508817, 13.05824, ...",[0.35200980659965037],1
3,"[-373.90222, 69.62015, 38.59025, 51.163128, 9....",[0.07336795691287878],1
4,"[-485.77536, 103.56779, 5.1006126, 12.516183, ...",[0.046856037621359224],1
...,...,...,...
161,"[-428.0815, 65.607445, -6.0010858, 17.54062, -...",[0.1625384706439394],5
162,"[-259.54175, 97.03256, -4.835197, 36.80339, -2...",[0.10187725981404959],5
163,"[-526.5556, 111.043015, 11.877811, 17.914228, ...",[0.028998013200431036],5
164,"[-368.36838, 79.430695, 18.874813, 32.134174, ...",[0.07103794642857143],5


In [317]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.2, random_state = 127)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(132, 3) (34, 3) (132, 1) (34, 1)


In [318]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [319]:
y_onehot_train = get_one_hot(y_train)
y_onehot_test = get_one_hot(y_test)
print(y_onehot_train.shape, y_onehot_test.shape)

(132, 5) (34, 5)


/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "cate

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [320]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
    return res

In [321]:
ls_train = get_processed_X(x_train)
ls_test = get_processed_X(x_test)
print(ls_train.shape, ls_test.shape)

(132, 41) (34, 41)


 # ნეირონული ქსელის კოდი

In [322]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [323]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [324]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [325]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 25
num_labels = y_onehot_train.shape[1]
learning_rate = 0.1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((25, 42), (5, 26))

In [326]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((132, 42), (132, 25), (132, 26), (132, 5), (132, 5))

In [327]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [329]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [330]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(3.6698525468479475, (1180,))

In [331]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 1.573547037860539
     jac: array([ 2.11144712e-08,  4.25791495e-05,  4.89281383e-05, ...,
       -1.68370391e-03, -1.58347896e-03, -5.48618581e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 39
  status: 3
 success: False
       x: array([ 0.00751297,  0.06307461,  0.06212658, ..., -0.56982676,
       -0.43751032, -0.07241006])

In [332]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
y_pred_train = np.array(np.argmax(h, axis=1) + 1)

In [333]:
ls_test = np.matrix(ls_test)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

In [334]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.6666666666666666


In [335]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.4411764705882353


# Feature selection
mfccs (learning rate = 1) = (40% - 50%)
mfccs + zero_crossing (lr = 1) = (~64%)
mfccs + zero_crossing + spectral_centroid = (~44%)
mfccs + zero_crossing + spectral_roll_ff = (~38%)
mfccs + zero_crossing + spectral_flux = (~23%)
mfccs + zero_crossing + chroma_features (~58%, ~87%(training))